In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip3 install -r ./drive/MyDrive/NLU\ Project/CODE/requirements.txt

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import transformers
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from datasets import load_dataset
import ast
import time
import json


#### Create helper functions key for data transformation and generating distractors

In [ ]:
def create_dataset(dataset_name = 'squad'):

  """
  1. Inputs dataset from huggingface website, convert it into a dataframe of Q,C,A (if not already done).
  2. Create 3 new columns for the 3 distractor answers.
  """

  dataset = load_dataset(dataset_name, split='train')
  questions = [q['question'] for q in dataset]
  contexts = [c['context'] for c in dataset]
  answers = [a['answers']['text'][0] if a['answers']['text'] else None for a in dataset]
  df = pd.DataFrame({
        'Question': questions,
        'Context': contexts,
        'Answer': answers,
        'Choice_1': [None] * len(dataset),
        'Choice_2': [None] * len(dataset),
        'Choice_3': [None] * len(dataset),
        'Choice_4': [None] * len(dataset)
    })

  return df

In [ ]:
def shuffle_dataset(new_dataset, shuffle):
  if shuffle == 0:
    new_dataset['Context'] = ''
  elif shuffle == 1:
    new_dataset['Shuffled Context'] = new_dataset['Context'].sample(frac=1).values
  return new_dataset

In [ ]:
def call_api(index, row):
    user_message = f"""context: {row['Context']}\n\
                    question: {row['Question']}\n\
                    answer: {row['Answer']}\n"""
    global client
    messages=[
        {"role": "system", "content": """You have a set of context, question and answer. Based on the question,\
                your task is to generate three distracting answers that look similar to the answer provided, but they\
                should be incorrect.
                Follow the provided structure when generating your output:
                {'A': 'distractor_1','B': 'distractor_2', 'C': 'distractor_3'}
                Make sure the three distractor answers are different from the answer provided by the user.
                Reference delimited with ### to outline real examples of user input of contexts, questions, answers, and expected outputs.
                ###
                {“role”: “user”, “content”, “””
                context: A Science Hall was built in 1883 under the direction of Fr. Zahm, but in 1950 it was converted to a student union building and named \
                LaFortune Center, after Joseph LaFortune, an oil executive from Tulsa, Oklahoma. Commonly known as "LaFortune" or "LaFun," it is a 4-story building \
                of 83,000 square feet that provides the Notre Dame community with a meeting place for social, recreational, cultural, and educational activities. \
                LaFortune employs 35 part-time student staff and 29 full-time non-student staff and has an annual budget of $1.2 million. Many businesses, services, \
                and divisions of The Office of Student Affairs are found within. The building also houses restaurants from national restaurant chains.\n\
                question: Which person oversaw the creation of a science hall at Notre Dame in 1883?\n\
                answer: Fr. Zahm\n“””
                }
                {“role”: “assistant”, “content”:
                      {'A': "Fr. Augustus",'B': "St. Geller",'C': "Rev. Greene"}
                }
                "
                ###
                ###
                {“role”: “user”, “content”, “””
                context:The New York Islanders and the New York Rangers represent the city in the National Hockey League. Also within the metropolitan area \
                are the New Jersey Devils, who play in nearby Newark, New Jersey.\n\
                question: There are two hockey teams located in NYC. What are they?\n\
                answer: The New York Islanders and the New York Rangers\n“””
                }
                {
                “role”: “assistant”, “content”:
                      {'A': "The New York Knicks and the Brooklyn Nets",'B': "The New York Yankees and the New York Mets",'C': "The New York Giants and the New York Sharks"}
                }
                ###
                ###
                {“role”: “user”, “content”, “””
                context:Guest judges may occasionally be introduced. In season two, guest judges such as Lionel Richie and Robin Gibb were used, and in season \
                three Donna Summer, Quentin Tarantino and some of the mentors also joined as judges to critique the performances in the final rounds. \
                Guest judges were used in the audition rounds for seasons four, six, nine, and fourteen such as Gene Simmons and LL Cool J in season four, \
                Jewel and Olivia Newton-John in season six, Shania Twain in season eight, Neil Patrick Harris, Avril Lavigne and Katy Perry in season nine, \
                and season eight runner-up, Adam Lambert, in season fourteen.\n\
                question: In what season was Lionel Richie a guest judge on American Idol?\n\
            `     answer: season two\n“””
                }
                {
                “role”: “assistant”, “content”:
                      {'A': "season three",'B': "season seven", 'C': "season six"}
                }
                ###
        """},

        {"role": "user", "content": user_message}
        ]

    attempts = 0
    while attempts < 3:
        try:
            response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
            content = response.choices[0].message.content
            content_dict = ast.literal_eval(content)
            return content_dict
        except SyntaxError as e:
            if e == 'invalid syntax (<unknown>, line 1)':
                print(f"Error for row: {index} \'{e}\'")
                return {"A": "Placeholder1", "B": "Placeholder2", "C": "Placeholder3"}
            print(f"Error for row: {index} {e}")
            attempts += 1
            if attempts < 3:
                print("Retrying in 1 minute...")
                time.sleep(60)
            else:
                print("Failed after 3 attempts.")
                return {"A": "Placeholder1", "B": "Placeholder2", "C": "Placeholder3"}

In [ ]:
def final_run(data, data_name):
    for index, row in data.iterrows():
        results = call_api(index, row)
        if results:
            data.at[index, 'Choice_1'] = results['A']
            data.at[index, 'Choice_2'] = results['B']
            data.at[index, 'Choice_3'] = results['C']
            print(f'Row {index} done')
            yield [results['A'], results['B'], results['C']]
        if index % 25 == 0:
            data.to_csv(f'updated_{data_name}.csv', index=False)
    data.to_csv(f'updated_{data_name}.csv', index=False)


In [ ]:
#Load original SQUAD dataset with 87k rows
df = create_dataset()

#Drop rows with duplicate context and keep top 10k rows
data_ = df.drop_duplicates(subset='Context', keep='first')
if len(data_) > 10000:
    data_ = data_.iloc[:10000]
else:
    data_ = data_

data_ = data_.reset_index(drop=True)

# Add the column for shuffled context
data_ = shuffle_dataset(data_, 1)

#### Trial 1: Test the API working for one row

In [ ]:
load_dotenv()
api_key = os.getenv('your-key')

In [ ]:
# Defaults to getting the key using os.environ.get("OPENAI_API_KEY")
# If you saved the key under a different environment variable name, you can do something like:
# client = OpenAI(
#   api_key=os.environ.get("CUSTOM_ENV_NAME"),
# )
client = OpenAI(api_key='your-key')


In [ ]:
# Sample message for one random row of data

messages=[
	{"role": "system", "content": """You have a set of context, question and answer. Based on the question,\
          your task is to generate three distracting answers that look similar to the answer provided, but they\
          should be incorrect.
          Follow the provided structure when generating your output:
          {'A': 'distractor_1','B': 'distractor_2', 'C': 'distractor_3'}
          Make sure the three distractor answers are different from the answer provided by the user.
          Reference delimited with ### to outline real examples of user input of contexts, questions, answers, and expected outputs.
          ###
          {“role”: “user”, “content”, “””
          context: A Science Hall was built in 1883 under the direction of Fr. Zahm, but in 1950 it was converted to a student union building and named \
          LaFortune Center, after Joseph LaFortune, an oil executive from Tulsa, Oklahoma. Commonly known as "LaFortune" or "LaFun," it is a 4-story building \
          of 83,000 square feet that provides the Notre Dame community with a meeting place for social, recreational, cultural, and educational activities. \
          LaFortune employs 35 part-time student staff and 29 full-time non-student staff and has an annual budget of $1.2 million. Many businesses, services, \
          and divisions of The Office of Student Affairs are found within. The building also houses restaurants from national restaurant chains.\n\
          question: Which person oversaw the creation of a science hall at Notre Dame in 1883?\n\
          answer: Fr. Zahm\n“””
          }
          {“role”: “assistant”, “content”:
                {'A': "Fr. Augustus",'B': "St. Geller",'C': "Rev. Greene"}
          }
          "
          ###
          ###
          {“role”: “user”, “content”, “””
          context:The New York Islanders and the New York Rangers represent the city in the National Hockey League. Also within the metropolitan area \
          are the New Jersey Devils, who play in nearby Newark, New Jersey.\n\
          question: There are two hockey teams located in NYC. What are they?\n\
          answer: The New York Islanders and the New York Rangers\n“””
          }
          {
          “role”: “assistant”, “content”:
                {'A': "The New York Knicks and the Brooklyn Nets",'B': "The New York Yankees and the New York Mets",'C': "The New York Giants and the New York Sharks"}
          }
          ###
          ###
          {“role”: “user”, “content”, “””
          context:Guest judges may occasionally be introduced. In season two, guest judges such as Lionel Richie and Robin Gibb were used, and in season \
          three Donna Summer, Quentin Tarantino and some of the mentors also joined as judges to critique the performances in the final rounds. \
          Guest judges were used in the audition rounds for seasons four, six, nine, and fourteen such as Gene Simmons and LL Cool J in season four, \
          Jewel and Olivia Newton-John in season six, Shania Twain in season eight, Neil Patrick Harris, Avril Lavigne and Katy Perry in season nine, \
          and season eight runner-up, Adam Lambert, in season fourteen.\n\
          question: In what season was Lionel Richie a guest judge on American Idol?\n\
          answer: season two\n“””
          }
          {
          “role”: “assistant”, “content”:
                {'A': "season three",'B': "season seven", 'C': "season six"}
          }
          ###
	"""},

	{"role": "user", "content": """context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden \
	statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ \
	with arms upraised with the legend 'Venite Ad Me Omnes'. Next to the Main Building is the Basilica of the Sacred Heart.\
	Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the \
	grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the \
	end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, \
	modern stone statue of Mary.\n\
	question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes, France?\n\
	answer: Saint Bernadette Soubirous\n
  """}
  ]

In [ ]:
completion = client.chat.completions.create(model="gpt-3.5-turbo", messages = messages)
print(completion.choices[0].message)

ChatCompletionMessage(content="{'A': 'Saint Catherine Labouré', 'B': 'Saint Teresa of Calcutta', 'C': 'Saint Joan of Arc'}", role='assistant', function_call=None, tool_calls=None)


In [ ]:
# This is how you access answers from the string encoded dictionary response
response = client.chat.completions.create(model="gpt-3.5-turbo", messages = messages)
content = response.choices[0].message.content
content_dict = ast.literal_eval(content)
print(content_dict['A'])
print(content_dict['B'])
print(content_dict['C'])

Saint Marguerite Bourgeoys
Saint Catherine Labouré
Saint Bernadine of Siena


#### Trial 2: If API works perefectly, next test the working of your function for populating distractors in 5 rows

In [ ]:
data_5= data_.sample(n=5)

In [ ]:
data_5

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
4967,When was the Jin Dynasty?,Following the fall of Wu and the turmoil of th...,265–420,None,None,None,None,"In the fall of 2014, among the six undergradua..."
5694,"In the field of immunology, what aspect is bec...",Immunological research continues to become mor...,research,None,None,None,None,The Thuringian Realm existed until 531 and lat...
7611,Who did Eden recommend that Elizabeth consult ...,The absence of a formal mechanism within the C...,Lord Salisbury,None,None,None,None,Drying involves using air and/or heat to remov...
4845,Axons grouped together are known as what?,Most of the space in the brain is taken up by ...,nerve fiber tracts,None,None,None,None,The Baylor College of Medicine has annually be...
9951,Are all algae different from land plants?,The algae are a polyphyletic group and are pla...,some more closely related,None,None,None,None,Several measurement methods have been develope...


In [ ]:
stored_responses_1 = {'A': [], 'B': [], 'C': []}

In [ ]:
stored_response_5 = final_run(data_5, 'data_5')

In [ ]:
stored_response_5

{'A': ['1955',
  'an integral part of the household',
  'Boundless',
  'LVMH Group',
  '1865'],
 'B': ['1950',
  'viewed as external companions',
  'Barriers',
  'Dalai Lama',
  '1869'],
 'C': ['1960', 'considered distant associates', 'Limitless', 'BOCOG', '1871']}

In [ ]:
df5 = pd.read_csv('updated_data_5.csv')

In [ ]:
# Print the dataframe to see if thats the result you expected!
df5

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,When was the bipolar transistor created?,The bipolar transistor was invented in 1947. F...,1947.,1955,1950,1960,NaN,^8 This identity continues to be used by a min...
1,How do most people describe the relationship w...,The majority of contemporary people with dogs ...,part of the family,an integral part of the household,viewed as external companions,considered distant associates,NaN,Neutering reduces problems caused by hypersexu...
2,What was the name of the song that Kris Allen ...,Both Allen and Lambert released the coronation...,No Boundaries,Boundless,Barriers,Limitless,NaN,"Matches are held within a wrestling ring, an e..."
3,Which French company was boycotted?,"In China, a call to boycott French hypermart C...",Carrefour,LVMH Group,Dalai Lama,BOCOG,NaN,Portuguese natives comprise a very small perce...
4,What year was Canada united in a confederation?,"Prior to Confederation in 1867, residents of t...",1867,1865,1869,1871,NaN,"In 2008, State Council established a counterpa..."


#### Once both Trials have run successfully, now you can implement it ont the full dataset (Still by making partitions)

In [ ]:
# Creating three splits of data to process them seperately.
data_1 = data_[0:3000]
data_2 = data_[3000:6000]
data_3 = data_[6000:]

In [ ]:
yielded_vals = final_run(data_1, 'data_1')
stored_responses_1 = {'A': [], 'B': [], 'C': []}

In [ ]:
for i in yielded_vals:
    stored_responses_1['A'].append(i[0])
    stored_responses_1['B'].append(i[1])
    stored_responses_1['C'].append(i[2])

Row 0 done
Row 1 done
Row 2 done
Row 3 done
Row 4 done
Row 5 done
Row 6 done
Row 7 done
Row 8 done
Row 9 done
Row 10 done
Row 11 done
Row 12 done
Row 13 done
Row 14 done
Row 15 done
Row 16 done
Row 17 done
Row 18 done
Row 19 done
Row 20 done
Row 21 done
Row 22 done
Row 23 done
Row 24 done
Row 25 done
Row 26 done
Row 27 done
Row 28 done
Row 29 done
Row 30 done
Row 31 done
Row 32 done
Row 33 done
Row 34 done
Row 35 done
Row 36 done
Row 37 done
Row 38 done
Row 39 done
Row 40 done
Row 41 done
Row 42 done
Row 43 done
Row 44 done
Row 45 done
Row 46 done
Row 47 done
Row 48 done
Row 49 done
Row 50 done
Row 51 done
Row 52 done
Row 53 done
Row 54 done
Row 55 done
Row 56 done
Row 57 done
Row 58 done
Row 59 done
Row 60 done
Row 61 done
Row 62 done
Row 63 done
Row 64 done
Row 65 done
Row 66 done
Row 67 done
Row 68 done
Row 69 done
Row 70 done
Row 71 done
Error for row: 72 invalid character '‘' (U+2018) (<unknown>, line 1)
Retrying in 2 minutes...
Row 72 done
Row 73 done
Row 74 done
Row 75 done
Row 

In [ ]:
df1 = pd.read_csv('updated_data_1.csv')

In [ ]:
df1

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,Saint Therese of Lisieux,Saint Joan of Arc,Saint Catherine of Siena,NaN,"In 2010, a genetic study was conducted on the ..."
1,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",September 1876,January 1872,August 1865,March 1880,NaN,"On November 2, 2014, the Cubs announced that J..."
2,Where is the headquarters of the Congregation ...,The university is the major seat of the Congre...,Rome,Jerusalem,Mecca,Athens,NaN,"In higher education, Politecnico refers to a t..."
3,How many BS level degrees are offered in the C...,The College of Engineering was established in ...,eight,five,two,four,NaN,While Southeast Asia is rich in flora and faun...
4,What entity provides help with the management ...,All of Notre Dame's undergraduate students are...,Learning Resource Center,Learning Support Facility,Career Development Center,Student Success Hub,NaN,Baird Television Ltd. made Britain's first tel...
...,...,...,...,...,...,...,...,...
2995,Who wrote the paper laying out hydrofoils and ...,The March 1906 Scientific American article by ...,William E. Meacham,William T. Johnson,William E. March,William A. Mitchell,NaN,The Lord Mayor's official residence is 3 Ellio...
2996,In what country did Baldwin and Forlanini get ...,"During his world tour of 1910–11, Bell and Bal...",France,Italy,Spain,Germany,NaN,The migration of birds also aids the movement ...
2997,What color silk covered Bell's kites?,"In 1898, Bell experimented with tetrahedral bo...",maroon,red,scarlet,crimson,NaN,Stress has a significant effect on memory form...
2998,What organization did Bell set up due to his i...,Bell was a supporter of aerospace engineering ...,Aerial Experiment Association,Aeronautical Testing Group,Flight Experiment Consortium,Aviation Research Society,NaN,Medieval writers divided history into periods ...


In [ ]:
yielded_vals = final_run(data_2, 'data_2')
stored_responses_2 = {'A': [], 'B': [], 'C': []}

In [ ]:
for i in yielded_vals:
    stored_responses_2['A'].append(i[0])
    stored_responses_2['B'].append(i[1])
    stored_responses_2['C'].append(i[2])

Row 3000 done
Row 3001 done
Error for row: 3002 invalid syntax (<unknown>, line 1)
Retrying in 1 minute...
Error for row: 3002 invalid syntax (<unknown>, line 1)
Retrying in 1 minute...
Error for row: 3002 invalid syntax (<unknown>, line 1)
Failed after 3 attempts.
Row 3002 done
Row 3003 done
Row 3004 done
Row 3005 done
Row 3006 done
Row 3007 done
Row 3008 done
Row 3009 done
Row 3010 done
Row 3011 done
Row 3012 done
Row 3013 done
Row 3014 done
Row 3015 done
Row 3016 done
Row 3017 done
Row 3018 done
Row 3019 done
Row 3020 done
Row 3021 done
Row 3022 done
Row 3023 done
Row 3024 done
Row 3025 done
Row 3026 done
Row 3027 done
Row 3028 done
Row 3029 done
Row 3030 done
Row 3031 done
Row 3032 done
Row 3033 done
Row 3034 done
Row 3035 done
Row 3036 done
Row 3037 done
Row 3038 done
Row 3039 done
Row 3040 done
Row 3041 done
Row 3042 done
Row 3043 done
Row 3044 done
Row 3045 done
Row 3046 done
Row 3047 done
Row 3048 done
Row 3049 done
Row 3050 done
Row 3051 done
Row 3052 done
Row 3053 done
Row 30

In [ ]:
df2 = pd.read_csv('updated_data_2.csv')

In [ ]:
df2

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,What did the AEA name their last plane?,"Their final aircraft design, the Silver Dart, ...",Silver Dart,Golden Falcon,Bronze Hawk,Platinum Eagle,NaN,During the second half of the 19th century Dut...
1,What kind of parents did Bell state were more ...,Bell was connected with the eugenics movement ...,congenitally deaf parents,both hearing parents,genetically modified parents,non-related parents,NaN,Besides earning a reputation as a respected en...
2,What are Bell's personal papers known as?,"A large number of Bell's writings, personal co...",Alexander Graham Bell Family Papers,Placeholder1,Placeholder2,Placeholder3,NaN,"In response to Cusumano's perspective, Screen ..."
3,"Adjusted for inflation, how many dollars were ...","In 1880, Bell received the Volta Prize with a ...","$250,000","$200,000","$220,000","$240,000",NaN,The city grew out towards the west of the boun...
4,What magazine did Bell found alongside Hubbard?,"In partnership with Gardiner Greene Hubbard, B...",Science,Technology,Physics,Chemistry,NaN,In many cities along the North American and Eu...
...,...,...,...,...,...,...,...,...
2995,"What are gender identity, ethnic identity, and...",The inclusiveness of Weinreich's definition (a...,one's total identity,one particular aspect of identity,partial components of identity,complete facets of identity,NaN,Most browsers support HTTP Secure and offer qu...
2996,What is distinct from the self?,"Although the self is distinct from identity, t...",identity,consciousness,ego,personality,NaN,Barcelona is one of the most supported teams i...
2997,Self-psychology is compelled to investigate ho...,"At a general level, self-psychology is compell...",the social environment,the natural world,the physical body,the spiritual realm,NaN,"According to the journal DesignIntelligence, w..."
2998,What term have Anthropologists employed to ref...,Anthropologists have most frequently employed ...,identity,selfhood,uniqueness,sameness,NaN,"On 7 July 1994, the Fraunhofer Society release..."


In [ ]:
yielded_vals = final_run(data_3, 'data_3')
stored_responses_3 = {'A': [], 'B': [], 'C': []}

In [ ]:
for i in yielded_vals:
    stored_responses_3['A'].append(i[0])
    stored_responses_3['B'].append(i[1])
    stored_responses_3['C'].append(i[2])

Row 6000 done
Row 6001 done
Row 6002 done
Row 6003 done
Row 6004 done
Row 6005 done
Row 6006 done
Row 6007 done
Row 6008 done
Row 6009 done
Row 6010 done
Row 6011 done
Row 6012 done
Row 6013 done
Row 6014 done
Row 6015 done
Row 6016 done
Row 6017 done
Row 6018 done
Row 6019 done
Row 6020 done
Row 6021 done
Row 6022 done
Row 6023 done
Row 6024 done
Row 6025 done
Row 6026 done
Row 6027 done
Row 6028 done
Row 6029 done
Row 6030 done
Row 6031 done
Row 6032 done
Row 6033 done
Row 6034 done
Row 6035 done
Row 6036 done
Row 6037 done
Row 6038 done
Row 6039 done
Row 6040 done
Row 6041 done
Row 6042 done
Row 6043 done
Row 6044 done
Row 6045 done
Row 6046 done
Row 6047 done
Row 6048 done
Row 6049 done
Row 6050 done
Row 6051 done
Row 6052 done
Row 6053 done
Row 6054 done
Row 6055 done
Row 6056 done
Row 6057 done
Row 6058 done
Row 6059 done
Row 6060 done
Row 6061 done
Row 6062 done
Row 6063 done
Row 6064 done
Row 6065 done
Row 6066 done
Row 6067 done
Row 6068 done
Row 6069 done
Row 6070 done
Row 60

In [ ]:
df3 = pd.read_csv('updated_data_3.csv')

In [ ]:
df3

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,What identity status paradigm emerged due to t...,"The ""Neo-Eriksonian"" identity status paradigm ...",Neo-Eriksonian,Marcian,Neomarcia,Ericsonian,NaN,Hinduism is the religion of 79.8% of New Delhi...
1,What do many people gain from their identity g...,Many people gain a sense of positive self-este...,a sense of positive self-esteem,a sense of positive self-image,a sense of self-doubt,a sense of negative self-esteem,NaN,Poland's AA defences were no match for the Ger...
2,What approach takes the sense of self and belo...,The first favours a primordialist approach whi...,a primordialist approach,a constructivist approach,a traditionalist approach,a conservative approach,NaN,"A fervent follower of the absolutist cause, El..."
3,Various research traditions are using what len...,The implications are multiple as various resea...,the lens of identity,the lens of perception,the lens of authenticity,the lens of diversity,NaN,Paper is often characterized by weight. In the...
4,What is it called when someone aspires to the ...,"However, the formation of one's identity occur...",benign,malign,defensive,contra-identification,NaN,"Wolves, and their dog descendants, would have ..."
...,...,...,...,...,...,...,...,...
3995,What was the film that Madonna starred in 2000?,"In 2000, Madonna starred in the film The Next ...",The Next Best Thing,The Last Great Thing,The Ultimate Solution,The Perfect Opportunity,NaN,Seattle is considered the home of grunge music...
3996,Who is Madonna's second husband?,"She met director Guy Ritchie, who would become...",Guy Ritchie,Director Smith,Actor Johnson,Producer Brown,NaN,The economic reforms of President Carlos Salin...
3997,Which film did Madonna starred in that was dir...,"Madonna starred in the film Swept Away, direct...",Swept Away,Up For Grabs,Die Another Day,Wyndhams Theatre,NaN,"Bell's patent 174,465, was issued to Bell on M..."
3998,What is the name of the exhibition installatio...,"Following Die Another Day, Madonna collaborate...",X-STaTIC Pro=CeSS,X-STiC Pro=CESS,X-STaTIX Pro=CeSS,X-CPXTIC Pro=CeSS,NaN,"The most pelagic species, mainly in the 'tuben..."


### Putting it all together + shuffling answers

In [ ]:
nlu_data= pd.concat([df1, df2, df3], axis=0, ignore_index=True)

In [ ]:
nlu_data

,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,Saint Therese of Lisieux,Saint Joan of Arc,Saint Catherine of Siena,NaN,"In 2010, a genetic study was conducted on the ..."
1,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",September 1876,January 1872,August 1865,March 1880,NaN,"On November 2, 2014, the Cubs announced that J..."
2,Where is the headquarters of the Congregation ...,The university is the major seat of the Congre...,Rome,Jerusalem,Mecca,Athens,NaN,"In higher education, Politecnico refers to a t..."
3,How many BS level degrees are offered in the C...,The College of Engineering was established in ...,eight,five,two,four,NaN,While Southeast Asia is rich in flora and faun...
4,What entity provides help with the management ...,All of Notre Dame's undergraduate students are...,Learning Resource Center,Learning Support Facility,Career Development Center,Student Success Hub,NaN,Baird Television Ltd. made Britain's first tel...
...,...,...,...,...,...,...,...,...
9995,What was the film that Madonna starred in 2000?,"In 2000, Madonna starred in the film The Next ...",The Next Best Thing,The Last Great Thing,The Ultimate Solution,The Perfect Opportunity,NaN,Seattle is considered the home of grunge music...
9996,Who is Madonna's second husband?,"She met director Guy Ritchie, who would become...",Guy Ritchie,Director Smith,Actor Johnson,Producer Brown,NaN,The economic reforms of President Carlos Salin...
9997,Which film did Madonna starred in that was dir...,"Madonna starred in the film Swept Away, direct...",Swept Away,Up For Grabs,Die Another Day,Wyndhams Theatre,NaN,"Bell's patent 174,465, was issued to Bell on M..."
9998,What is the name of the exhibition installatio...,"Following Die Another Day, Madonna collaborate...",X-STaTIC Pro=CeSS,X-STiC Pro=CESS,X-STaTIX Pro=CeSS,X-CPXTIC Pro=CeSS,NaN,"The most pelagic species, mainly in the 'tuben..."


In [ ]:
#Test for any NaN values in the the three new columns
na_count_1 = nlu_data['Choice_1'].isna().sum()
print(na_count_1)

na_count_2 = nlu_data['Choice_2'].isna().sum()
print(na_count_2)

na_count_3 = nlu_data['Choice_3'].isna().sum()
print(na_count_3)

0
1
0


In [ ]:
# There is one NaN value in Choice_2, based on inspectiion we find out that it is actually not an error. Hence we replace it with another disrupter answer.
nan_rows_choice_2 = nlu_data[nlu_data['Choice_2'].isna()]
print(nan_rows_choice_2)
nlu_data['Choice_2'] = nlu_data['Choice_2'].fillna('Missing')